<a href="https://colab.research.google.com/github/jtfreitas/QInfo_Final/blob/main/tensornetwork_play.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip3 install tensornetwork jax jaxlib -q #--quiet flag
!wget -O RNA_cancer.tar.gz  https://archive.ics.uci.edu/ml/machine-learning-databases/00401/TCGA-PANCAN-HiSeq-801x20531.tar.gz
!tar -xvzf RNA_cancer.tar.gz
!mv TCGA-PANCAN-HiSeq-801x20531 RNA_data

     |████████████████████████████████| 364 kB 8.5 MB/s 
     |████████████████████████████████| 46 kB 2.9 MB/s 
--2022-02-13 08:03:44--  https://archive.ics.uci.edu/ml/machine-learning-databases/00401/TCGA-PANCAN-HiSeq-801x20531.tar.gz
Resolving archive.ics.uci.edu (archive.ics.uci.edu)... 128.195.10.252
Connecting to archive.ics.uci.edu (archive.ics.uci.edu)|128.195.10.252|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 72856320 (69M) [application/x-httpd-php]
Saving to: ‘RNA_cancer.tar.gz’

RNA_cancer.tar.gz   100%[===================>]  69.48M  31.5MB/s    in 2.2s    

2022-02-13 08:03:47 (31.5 MB/s) - ‘RNA_cancer.tar.gz’ saved [72856320/72856320]

TCGA-PANCAN-HiSeq-801x20531/
TCGA-PANCAN-HiSeq-801x20531/data.csv
TCGA-PANCAN-HiSeq-801x20531/labels.csv


In [23]:
import tensornetwork as tn
import numpy as np
import tensorflow as tf
from tensorflow.keras.datasets import mnist
from tensorflow.keras.layers import Dense
import time
import pandas as pd

tn.set_default_backend('tensorflow')

In [ ]:
(x_train, y_train), (x_test, y_test) = mnist.load_data('/content/mnist.npz')

y_train = tf.one_hot(y_train, 10)
in_shape = x_train[0].shape

In [ ]:
def make_vector(pixel):
  return np.array[np.cos(np.pi/2*pixel), np.sin(np.pi/2*pixel)]

def feature_map(data_point):
  feature_map = np.vectorize(make_vector)(data_point)

  return feature_map

#cluster average function, will be used later for generating the new_x, so to train over variations of the data?
def avg_cluster(data, size):
  out_shape = (data.shape[1]//size, data.shape[2]//size)
  clustered_sample = np.zeros(out_shape)
  for i in range(0, out_shape[0]):
    for j in range(0, out_shape[1]):
      clustered_sample[i,j] = np.mean(sample[size*i:size*(i+1), size*j:size*(j+1)])
  return clustered_sample

# cluster = np.vectorize(avg_cluster)
x_train0 = np.zeros((len(x_train), 14, 14))
for i, sample in enumerate(x_train):
  x_train0[i] = avg_cluster(sample, 2)

x_train = x_train0

SyntaxError: ignored

In [ ]:
def make_vector(pixel):
  return [np.cos(np.pi/2*pixel), np.sin(np.pi/2*pixel)]

def feature_map(data_point):
  feature_map = np.vectorize(make_vector)(data_point)

  return feature_map

trans_x_train = np.array([[make_vector(pixel) for pixel in data_point.flatten()] for data_point in x_train0])

In [3]:
#Building the tensor network

class TNLayer(tf.keras.layers.Layer):

  def __init__(self):
    super(TNLayer, self).__init__()
    # Create the variables for the layer.
    self.a_var = tf.Variable(tf.random.normal(shape=(32,12,10),
                                              stddev=1.0/32.0),
                             name="a", trainable=True)
    self.b_var = tf.Variable(tf.random.normal(shape=(32,336,10),
                                              stddev=1.0/32.0),
                             name="b", trainable=True)
    self.bias = tf.Variable(tf.zeros(shape=(32,32)),
                            name="bias", trainable=True)

  def call(self, inputs):
    # Define the contraction.
    # We break it out so we can parallelize a batch using
    # tf.vectorized_map (see below).
    def f(input_vec, a_var, b_var, bias_var):
      # Reshape to a matrix instead of a vector.
      input_vec = tf.reshape(input_vec, (12,336))

      # Now we create the network.
      a = tn.Node(a_var)
      b = tn.Node(b_var)
      x_node = tn.Node(input_vec)
      a[1] ^ x_node[0]
      b[1] ^ x_node[1]
      a[2] ^ b[2]

      # The TN should now look like this
      #   |     |
      #   a --- b 
      #    \   /
      #      x

      # Now we begin the contraction.
      biggie = a @ x_node
      result = (biggie @ b).tensor

      # Finally, add bias.
      return result + bias_var

    # To deal with a batch of items, we can use the tf.vectorized_map
    # function.
    # https://www.tensorflow.org/api_docs/python/tf/vectorized_map
    result = tf.vectorized_map(
        lambda vec: f(vec, self.a_var, self.b_var, self.bias), inputs)
    return tf.nn.relu(tf.reshape(result, (-1, 1024)))

In [ ]:
Dense = tf.keras.layers.Dense
Conv2D = tf.keras.layers.Conv2D
MaxPooling2D = tf.keras.layers.MaxPooling2D
Flatten = tf.keras.layers.Flatten
tn_model = tf.keras.Sequential(
    [
     tf.keras.Input(shape=(*in_shape,1,)),
     Conv2D(28, (4,4), strides = (1,1)),
     MaxPooling2D((2,2)),
     Flatten(),
   #  Dense(1024, activation=tf.nn.relu),
     # Here, we replace the dense layer with our MPS.
     TNLayer(),
     Dense(10, activation='softmax')])
tn_model.summary()

tn_model.compile(optimizer = 'adam', loss = 'categorical_crossentropy', metrics = ['accuracy'])

Model: "sequential_6"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_6 (Conv2D)           (None, 25, 25, 28)        476       
                                                                 
 max_pooling2d_6 (MaxPooling  (None, 12, 12, 28)       0         
 2D)                                                             
                                                                 
 flatten_6 (Flatten)         (None, 4032)              0         
                                                                 
 tn_layer_6 (TNLayer)        (None, 1024)              112384    
                                                                 
 dense_6 (Dense)             (None, 10)                10250     
                                                                 
Total params: 123,110
Trainable params: 123,110
Non-trainable params: 0
________________________________________________

In [ ]:
hist_dict = tn_model.fit(x_train, y_train, batch_size = 4096, epochs = 10, verbose = 1, validation_split = 0.2, shuffle=True)

Epoch 1/10
12/12 [==============================] - 90s 7s/step - loss: 2.7376 - accuracy: 0.5385 - val_loss: 0.7091 - val_accuracy: 0.8148
Epoch 2/10
12/12 [==============================] - 80s 7s/step - loss: 0.5519 - accuracy: 0.8521 - val_loss: 0.4066 - val_accuracy: 0.8939
Epoch 3/10
12/12 [==============================] - 80s 7s/step - loss: 0.3520 - accuracy: 0.9013 - val_loss: 0.2916 - val_accuracy: 0.9215
Epoch 4/10
12/12 [==============================] - 80s 7s/step - loss: 0.2612 - accuracy: 0.9231 - val_loss: 0.2299 - val_accuracy: 0.9333
Epoch 5/10
12/12 [==============================] - 80s 7s/step - loss: 0.2120 - accuracy: 0.9372 - val_loss: 0.2003 - val_accuracy: 0.9427
Epoch 6/10
12/12 [==============================] - 80s 7s/step - loss: 0.1816 - accuracy: 0.9462 - val_loss: 0.1808 - val_accuracy: 0.9483
Epoch 7/10
12/12 [==============================] - 80s 7s/step - loss: 0.1598 - accuracy: 0.9527 - val_loss: 0.1650 - val_accuracy: 0.9520
Epoch 8/10
12/12 [==

In [ ]:
y_test = tf.one_hot(y_test, 10)

tn_model.evaluate(x_test, y_test)

313/313 [==============================] - 25s 79ms/step - loss: 0.1375 - accuracy: 0.9582


[0.13754282891750336, 0.9581999778747559]

In [4]:
data = pd.read_csv('/content/RNA_data/data.csv')
labels = pd.read_csv('/content/RNA_data/labels.csv')

In [5]:
X = data[data.keys()[1:]].to_numpy()

In [6]:
def translate(label, trans_dict):
  return trans_dict[label]


translation = {label : i for i, label in enumerate(labels['Class'].unique())}
y_converted = labels['Class'].apply(translate, args=(translation,))
y = y_converted.to_numpy()

In [ ]:
tf.one_hot(y, 5)

<tf.Tensor: shape=(801, 5), dtype=float32, numpy=
array([[1., 0., 0., 0., 0.],
       [0., 1., 0., 0., 0.],
       [1., 0., 0., 0., 0.],
       ...,
       [0., 0., 0., 0., 1.],
       [1., 0., 0., 0., 0.],
       [1., 0., 0., 0., 0.]], dtype=float32)>

In [7]:
#Building the tensor network
class TNLayer(tf.keras.layers.Layer):

  def __init__(self):
    super(TNLayer, self).__init__()
    # Create the variables for the layer.
    self.a_var = tf.Variable(tf.random.normal(shape=(32,419,5),
                                              stddev=1.0/32.0),
                             name="a", trainable=True)
    self.b_var = tf.Variable(tf.random.normal(shape=(32,7,5,5),
                                              stddev=1.0/32.0),
                             name="b", trainable=True)
    self.c_var = tf.Variable(tf.random.normal(shape=(12,7,5),
                                         stddev=1.0/32.0),
                         name="c", trainable=True)
    self.bias = tf.Variable(tf.zeros(shape=(32,32,12)),
                            name="bias", trainable=True)

  def call(self, inputs):
    # Define the contraction.
    def f(input_vec, a_var, b_var, c_var, bias_var):

      # The TN aspect
      #   |     |     |
      #   a --- b --- c
      #    \    |    /
      #      ---x---

      # Reshape input to matrix.
      input_vec = tf.reshape(input_vec, (419,7,7))


      result = tn.ncon([input_vec, a_var, b_var, c_var], [[1, 2, 3], [-1, 1, 4], [-2, 2, 4, 5], [-3,3,5]])
      #Numbers here are edge numbers. Their position must match the index positions
      # Finally, add bias.
      return result + bias_var

    # To deal with a batch of items, we can use the tf.vectorized_map
    # function.
    # https://www.tensorflow.org/api_docs/python/tf/vectorized_map
    result = tf.vectorized_map(
        lambda vec: f(vec, self.a_var, self.b_var, self.c_var, self.bias), inputs)
    return tf.nn.relu(tf.reshape(result, (-1, 12288))) #perform activation

In [24]:
X_train, y_train = X[:680], y[:680]
X_test, y_test = X[680:], y[680:]

y_train, y_test = tf.one_hot(y_train, 5), tf.one_hot(y_test, 5)

RNA_tn = tf.keras.Sequential(
    [tf.keras.Input(20531, batch_size=136),
     TNLayer(),
     Dense(5, activation='softmax')
     ]
)
RNA_tn.summary()
RNA_tn.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

Model: "sequential_4"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 tn_layer_3 (TNLayer)        (136, 12288)              85348     
                                                                 
 dense_6 (Dense)             (136, 5)                  61445     
                                                                 
Total params: 146,793
Trainable params: 146,793
Non-trainable params: 0
_________________________________________________________________


In [25]:
start = time.time()
history = RNA_tn.fit(X_train, y_train, batch_size=136, epochs=20, validation_split=0.2, shuffle=True)
end = time.time()
print(end - start)

Epoch 1/20
4/4 [==============================] - 2s 307ms/step - loss: 1.6297 - accuracy: 0.3640 - val_loss: 1.4290 - val_accuracy: 0.6544
Epoch 2/20
4/4 [==============================] - 1s 192ms/step - loss: 1.3110 - accuracy: 0.5864 - val_loss: 1.1873 - val_accuracy: 0.3676
Epoch 3/20
4/4 [==============================] - 1s 193ms/step - loss: 1.0868 - accuracy: 0.6085 - val_loss: 0.9533 - val_accuracy: 0.8824
Epoch 4/20
4/4 [==============================] - 1s 190ms/step - loss: 0.8345 - accuracy: 0.8548 - val_loss: 0.6879 - val_accuracy: 0.8235
Epoch 5/20
4/4 [==============================] - 1s 188ms/step - loss: 0.5737 - accuracy: 0.9228 - val_loss: 0.4482 - val_accuracy: 1.0000
Epoch 6/20
4/4 [==============================] - 1s 193ms/step - loss: 0.3512 - accuracy: 0.9890 - val_loss: 0.2476 - val_accuracy: 0.9779
Epoch 7/20
4/4 [==============================] - 1s 192ms/step - loss: 0.1849 - accuracy: 0.9945 - val_loss: 0.1378 - val_accuracy: 1.0000
Epoch 8/20
4/4 [====

In [26]:
RNA_tn.evaluate(X_test, y_test, batch_size=121)

1/1 [==============================] - 0s 394ms/step - loss: 0.0087 - accuracy: 1.0000


[0.00866545271128416, 1.0]

In [29]:
RNA_classic = tf.keras.Sequential(
    [tf.keras.Input(20531, batch_size=136),
     Dense(1024, activation='relu'),
     Dense(5, activation='softmax')
     ]
)
RNA_classic.summary()
RNA_classic.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

Model: "sequential_6"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_9 (Dense)             (136, 1024)               21024768  
                                                                 
 dense_10 (Dense)            (136, 5)                  5125      
                                                                 
Total params: 21,029,893
Trainable params: 21,029,893
Non-trainable params: 0
_________________________________________________________________


In [30]:
start = time.time()
hist_classic = RNA_classic.fit(X_train, y_train, batch_size = 136, epochs=20, validation_split = 0.2, shuffle=True)
end = time.time()
print(end - start)

Epoch 1/20
4/4 [==============================] - 2s 318ms/step - loss: 802.6733 - accuracy: 0.2500 - val_loss: 1303.4558 - val_accuracy: 0.1176
Epoch 2/20
4/4 [==============================] - 1s 269ms/step - loss: 790.2600 - accuracy: 0.1213 - val_loss: 130.8927 - val_accuracy: 0.1765
Epoch 3/20
4/4 [==============================] - 1s 266ms/step - loss: 124.9977 - accuracy: 0.3419 - val_loss: 123.4587 - val_accuracy: 0.3676
Epoch 4/20
4/4 [==============================] - 1s 269ms/step - loss: 114.6471 - accuracy: 0.3309 - val_loss: 63.7763 - val_accuracy: 0.3529
Epoch 5/20
4/4 [==============================] - 1s 267ms/step - loss: 76.8782 - accuracy: 0.3180 - val_loss: 54.7279 - val_accuracy: 0.3676
Epoch 6/20
4/4 [==============================] - 1s 268ms/step - loss: 43.0847 - accuracy: 0.5037 - val_loss: 57.7767 - val_accuracy: 0.3382
Epoch 7/20
4/4 [==============================] - 1s 265ms/step - loss: 29.6157 - accuracy: 0.5533 - val_loss: 1.2997 - val_accuracy: 0.8309

In [31]:
RNA_classic.evaluate(X_test, y_test, batch_size=121)

1/1 [==============================] - 0s 210ms/step - loss: 0.1495 - accuracy: 0.9917


[0.14948438107967377, 0.9917355179786682]